In [7]:
#environment setting
import pandas as pd
import sqlite3
import re
# Library Import and Environment Setting needed for Logistic Regression

import numpy as np
import matplotlib.pyplot as plt 
from matplotlib.backends.backend_pdf import PdfPages
%matplotlib inline

plt.rc("font", size=16)
import seaborn as sns
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)

import os
import os.path
import glob
import re

import json

from collections import defaultdict
from collections import Counter

from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer

! pip install profanity-check
from profanity_check import predict, predict_prob
# https://github.com/vzhou842/profanity-check

In [3]:
os.chdir('/Users/alexpapaioannou/Documents/GitHub/TFA_Project/')

raw_filenames = []
artists = []

# Get .txt files names
for file_name in os.listdir('Lyrics'):
    if file_name.endswith('.txt'):
        raw_filenames.append(file_name)
        artists.append(file_name.split("~")[2].split(".txt")[0])

def raw_filenames_list(str)->list:
    raw_filenames_list_ = []
    for filename in os.listdir('Lyrics'):
        if filename.endswith(str):
            raw_filenames_list_.append(filename)
    return raw_filenames_list_

# print(raw_filenames_list('.txt'))     # uncomment to see the function's result
# print(len(raw_filenames_list('.txt')))# uncomment to see the function's result

def artists_list(str)->list:
    artists_list_ = []
    for filename in raw_filenames:
        if filename.endswith(str):
            artists_list_.append(filename.split("~")[2].split(".txt")[0])
    return artists_list_

# print(artists_list('.txt'))          # uncomment to see the function's result
# print(len(artists_list('.txt')))     # uncomment to see the len (function's result)
## print(set(artists_list('.txt')))    # uncomment to see the function's result in the set(artists)
# print(len(set(artists_list('.txt'))))# uncomment to see the the len(function's result in the set(artists))

def songs_list(str)->list:
    songs_list_ = []
    for filename in raw_filenames:
        if filename.endswith(str):
            songs_list_.append(filename.split("~")[1].split(".txt")[0])
    return songs_list_

# print(songs_list('.txt'))              # uncomment to see the function's result
# print(len(songs_list('.txt')))         # uncomment to see the len (function's result)

def song_ordering_numbers_list(str)->list:
    song_ordering_numbers_list_ = []
    for filename in raw_filenames:
        if filename.endswith(str):
            song_ordering_numbers_list_.append(filename.split("~")[0].split(".txt")[0])
    return song_ordering_numbers_list_

# print(song_ordering_numbers_list('.txt'))       # uncomment to see the function's result
# print(len(song_ordering_numbers_list('.txt')))  # uncomment to see the len (function's result)

def artist_s_songs_list(str)->list:
    artist_s_songs_list_ = []
    for raw_filename in os.listdir('Lyrics'):
        if raw_filename.endswith(str+'.txt'):
#           print("For the artist", str, "the file(s)", raw_filenames, "has the keyword")
            artist_s_songs_list_.append(raw_filename)
#           print(artist_s_songs_list_)
    return artist_s_songs_list_

# for artist in set(artists):               # uncomment to see the function's result
#     print(artist_s_songs_list(artist))    # uncomment to see the function's result

In [4]:
for artist in set(artists):
    for song in artist_s_songs_list(artist):
        f = open('Lyrics/' + song, 'rb')
        all_words = ''
        for sentence in f.readlines():
            this_sentence = sentence.decode('utf-8')
            all_words += this_sentence
        #remove identifiers like chorus, verse, etc
        all_words = re.sub(r'[\(\[],.*?[\)\]]', '', all_words)
        #remove empty lines
        all_words = os.linesep.join([s for s in all_words.splitlines() if s])
        f.close()
        directory = "Lyrics/Cleaned_Songs"
        f = open(os.path.join(directory, 'cleaned_' + song ), "wb")
        f.write(all_words.encode('utf-8'))
        f.close()

In [5]:
plt.rcParams['figure.figsize'] = (126,190)

df = pd.DataFrame(columns=('artist', 'words'))
i=0
            
def artist_s_cleaned_songs_list(str)->list:
    artist_s_cleaned_songs_list_ = []
    for cleaned_raw_filename in os.listdir("Lyrics/Cleaned_Songs/"):
        if cleaned_raw_filename.endswith(str + '.txt') is True:
            if cleaned_raw_filename.endswith(str + '.txt') not in artist_s_cleaned_songs_list_:
                artist_s_cleaned_songs_list_.append(cleaned_raw_filename)
    return artist_s_cleaned_songs_list_


# for artist in set(artists):                                     # uncomment to see the function's result
#     print(artist, '\n', artist_s_cleaned_songs_list(artist))    # uncomment to see the function's result

for artist in set(artists):
    num_words = 0
    all_words = ''
    for song in artist_s_cleaned_songs_list(artist):
        f = open('Lyrics/Cleaned_Songs/' + song , 'rb')
        for sentence in f.readlines():
            this_sentence = sentence.decode('utf-8')
            num_words_this = len(this_sentence.split(" "))
            num_words += num_words_this


    df.loc[i] = (artist, num_words)
    i+=1

df.plot.bar(x='artist', y='words', title='Number of Words for each Artist');
plt.savefig('Number_of_Words_for_each_Artist_result.pdf')
plt.show()

In [5]:
df = pd.DataFrame(columns=('artist', 'pos', 'neu', 'neg'))
sid = SentimentIntensityAnalyzer()
# Commas do not affect the SentimentIntensityAnalyzer() (http://www.nltk.org/howto/sentiment.html)
i=0

os.chdir('/Users/alexpapaioannou/Documents/GitHub/TFA_Project/')

num_neutral = 0
num_negative = 0
num_positive = 0
    
for artist in set(artists):
    
    num_neutral = 0
    num_negative = 0
    num_positive = 0

    for song in artist_s_cleaned_songs_list(artist):
        f = open('Lyrics/Cleaned_Songs/' + song , 'rb')
        for sentence in f.readlines():
            this_sentence = sentence.decode('utf-8')
            comp = sid.polarity_scores(this_sentence)
            comp = comp['compound']
            if comp >= 0.5:
                num_positive += 1
            elif comp > -0.5 and comp < 0.5:
                num_neutral += 1
            else:
                num_negative += 1
                
    num_total = num_negative + num_neutral + num_positive
    if num_total != 0:
        percent_negative = (num_negative/float(num_total))*100
        percent_neutral = (num_neutral/float(num_total))*100
        percent_positive = (num_positive/float(num_total))*100
        df.loc[i] = (artist, percent_positive, percent_neutral, percent_negative)
        i+=1
    else:
        percent_negative = 0
        percent_neutral = 0
        percent_positive = 0
        df.loc[i] = (artist, percent_positive, percent_neutral, percent_negative)
        i+=1

df.plot.bar(x='artist', stacked=True)
# plt.tick_params(axis = 'x', direction='out', length=26, width=49)
plt.savefig('Sentiment_breakdown_for_each_Artist_result.pdf')
plt.show()

In [6]:
plt.rcParams['figure.figsize'] = (126,190)

df = pd.DataFrame(columns=('artist', 'words'))
i=0
            
def artist_s_cleaned_songs_list(str)->list:
    artist_s_cleaned_songs_list_ = []
    for cleaned_raw_filename in os.listdir("Lyrics/Cleaned_Songs/"):
        if cleaned_raw_filename.endswith(str + '.txt') is True:
            if cleaned_raw_filename.endswith(str + '.txt') not in artist_s_cleaned_songs_list_:
                artist_s_cleaned_songs_list_.append(cleaned_raw_filename)
    return artist_s_cleaned_songs_list_


# for artist in set(artists):                                    # uncomment to see the function's result
#     print(artist, '\n', artist_s_cleaned_songs_list(artist))   # uncomment to see the function's result

num_words_list_ = []

for artist in set(artists):
    num_words = 0
    for song in artist_s_cleaned_songs_list(artist):
        f = open('Lyrics/Cleaned_Songs/' + song , 'rb')
        for sentence in f.readlines():
            this_sentence = sentence.decode('utf-8')
            num_words_this = len(this_sentence.split(" "))
            num_words += num_words_this
    num_words_list_.append(num_words)
#     df.loc[i] = (artist, num_words)                             # uncomment for visualizations
#     i+=1                                                        # uncomment for visualizations

min_num_words_list_ = min(num_words_list_)
max_num_words_list_ = max(num_words_list_)
# df.plot.bar(x='artist', y='words', title='Number of Words for each Artist'); # uncomment for visualizations
# plt.savefig('Number_of_Words_for_each_Artist_result.pdf')                    # uncomment for visualizations
# plt.show()                                                                   # uncomment for visualizations
    
# print(num_words_list_)      # uncomment to see the list with the number of words per song
# print(min(num_words_list_)) # uncomment to see the minimum of the list with the number of words per song
# print(max(num_words_list_)) # uncomment to see the maximum of the list with the number of words per song

In [7]:
# Creating the regularization step for the length attribute

for artist in set(artists):
    num_words = 0
    for song in artist_s_cleaned_songs_list(artist):
        f = open('Lyrics/Cleaned_Songs/' + song , 'rb')
        for sentence in f.readlines():
            this_sentence = sentence.decode('utf-8')
            num_words_this = len(this_sentence.split(" "))
            num_words += num_words_this
        regularization_step = (num_words - min_num_words_list_)/(max_num_words_list_ - min_num_words_list_) 
        num_words_score = 1*regularization_step + 0*(1-regularization_step)
        num_words_score = round(num_words_score,2)
#         print(num_words_score) # uncomment to see the complexity of each song

In [8]:
plt.rcParams['figure.figsize'] = (126,190)

df = pd.DataFrame(columns=('artist', 'lexicalrichness'))
for artist in set(artists):
    num_words = 0
    all_words = ''
    raw_text = ""
    for song in artist_s_cleaned_songs_list(artist):
        f = open('Lyrics/Cleaned_Songs/' + song , 'rb')
        for sentence in f.readlines():
            this_sentence = sentence.decode('utf-8')
            raw_text += this_sentence
            
    words = raw_text.split(" ")
    filtered_words = [word for word in words if word not in stopwords.words('english') and len(word) > 1 and word not in ['na','la']] # remove the stopwords
    
    a = len(set(filtered_words))
    b = len(words)
    df.loc[i] = (artist, (a/float(b))*100)                      # uncomment for visualizations
    i+=1                                                        # uncomment for visualizations
    
df.plot.bar(x='artist', y='lexicalrichness', title='Lexical richness of each Artist'); # uncomment for visualizations
plt.savefig('Lexical_Richness_for_each_Artist_result.pdf')                             # uncomment for visualizations
plt.show()                                                                             # uncomment for visualizations

unique_words_list_ = []

for artist in set(artists):
    num_words = 0
    all_words = ''
    raw_text = ""    
    for song in artist_s_cleaned_songs_list(artist):
        f = open('Lyrics/Cleaned_Songs/' + song , 'rb')
        for sentence in f.readlines():
            this_sentence = sentence.decode('utf-8')
            raw_text += this_sentence
    words = raw_text.split(" ")
    filtered_words = [word for word in words if word not in stopwords.words('english') and len(word) > 1 and word not in ['na','la']] # remove the stopwords
    
    a = len(set(filtered_words))
    b = len(words)
    unique_words_ratio = a/float(b)
    unique_words_ratio = round(unique_words_ratio,2)
    unique_words_list_.append(unique_words_ratio)
    
#     df.loc[i] = (artist, (a/float(b))*100)                      # uncomment for visualizations
#     i+=1                                                        # uncomment for visualizations

min_unique_words_list_ = min(unique_words_list_)                # not needed
max_unique_words_list_ = max(unique_words_list_)                # not needed
 
# df.plot.bar(x='artist', y='lexicalrichness', title='Lexical richness of each Artist'); # uncomment for visualizations
# plt.savefig('Lexical_Richness_for_each_Artist_result.pdf')                             # uncomment for visualizations
# plt.show()    

print(unique_words_list_)      # uncomment to see the list with the unique words per song
print(min(unique_words_list_)) # uncomment to see the minimum of the list with the unique words per song
print(max(unique_words_list_)) # uncomment to see the maximum of the list with the unique words per song

[0.31, 0.63, 0.75, 0.26, 0.3, 0.43, 0.23, 0.31, 0.45, 0.29, 0.27, 0.58, 0.5, 0.38, 0.47, 0.3, 0.52, 0.67, 0.28, 0.27, 0.38, 0.41, 0.36, 0.34, 0.37, 0.44, 0.51, 0.61, 0.55, 0.23, 0.45, 0.24, 0.28, 0.45, 0.41, 0.4, 0.62, 0.16, 0.48, 0.49, 0.47, 0.25, 0.49, 0.27, 0.42, 0.3, 0.45, 0.21, 0.37, 0.38, 0.21, 0.47, 0.27, 0.15, 0.3, 0.28, 0.29, 0.42, 0.47, 0.37, 0.4, 0.29, 0.51, 0.59, 0.57, 0.49, 0.32, 0.43, 0.42, 0.6, 0.59, 0.29, 0.37, 0.21, 0.58, 0.72, 0.42, 0.58, 0.51, 0.36, 0.27, 0.35, 0.45, 0.59, 0.51, 0.43, 0.21, 0.74, 0.43, 0.24, 0.45, 0.45, 0.39, 0.34, 0.32, 0.45, 0.73, 0.42, 0.6, 0.41, 0.48, 0.26, 0.28, 0.36, 0.51, 0.34, 0.21, 0.36, 0.52, 0.56, 0.47, 0.36, 0.4, 0.4, 0.34, 0.49, 0.42, 0.47, 0.25, 0.36, 0.42, 0.33, 0.41, 0.28, 0.78, 0.32, 0.38, 0.43, 0.43, 0.22, 0.42, 0.5, 0.56, 0.32, 0.5, 0.3, 0.34, 0.29, 0.25, 0.42, 0.33, 0.5, 0.27, 0.58, 0.69, 0.23, 0.35, 0.26, 0.48, 0.4, 0.51, 0.27, 0.48, 0.44, 0.33, 0.38, 0.23, 0.25, 0.45, 0.46, 0.37, 0.59, 0.48, 0.37, 0.44, 0.28, 0.47, 0.43, 0.34, 0

In [9]:
num_neutral = 0
num_negative = 0
num_positive = 0
unique_words_list_ = []
love_words_list_ = [
                   'adore', 'adores', 'adorable', 'affection', 'amour', 'angel', 'bliss', 
#                    'care', 'caring', 'chocolate', 'companion', 'compassion', 'concern', 
#                    'darling', 'dear', 'desire', 'devotion', 'endearment', 'family', 
#                    'fondness', 'forever', 'friendship', 'fun', 'God', 'happiness', 'happy', 
                   'happily', 'heart', 'hugs', 'husband', 'infatuation', 'inspiration', 
                   'intimacy', 'joy', 'kiss', 'kisses', 
                   'love', 'loves', 'loved', 'loving', 
#                    'loyalty', 'marriage', 'passion', 'relationship', 'romance', 'sex', 
#                    'sweet', 'sweetheart', 'tenderness', 'trust', 'warmth', 'wife'
                    ]

love_ratio_list_ = []
dict_for_json = {}

with open('./data_output.txt', 'w') as f:

    for artist in artists:   

        num_neutral = 0
        num_negative = 0
        num_positive = 0
        
        num_words = 0
        all_words = ''
        raw_text = "" 
        
        counter_for_love_words = 0

        for song in artist_s_cleaned_songs_list(artist):
            cleaned_song_name = song.split("cleaned_")[1].split("~")[1]
            id_ = song.split("cleaned_")[1].split("~")[0]
            f = open('Lyrics/Cleaned_Songs/' + song , 'rb')
            for sentence in f.readlines():
                this_sentence = sentence.decode('utf-8')
                
                raw_text += this_sentence
            
            words = raw_text.split(" ")
            filtered_words = [word for word in words if word not in stopwords.words('english') and len(word) > 1 and word not in ['na','la']] # remove the stopwords
            
            for item in words:
                if item in love_words_list_:
                    counter_for_love_words += 1
            len_counter_for_love_words = len(love_words_list_)
            love_ratio = counter_for_love_words/float(len_counter_for_love_words)
            love_ratio = round(love_ratio,4)
            love_ratio_list_.append(love_ratio) 
            
min_love_ratio_list_ = min(love_ratio_list_)
max_love_ratio_list_ = max(love_ratio_list_)

print(love_ratio_list_)      # uncomment to see the list with the number of words per song
print(min(love_ratio_list_)) # uncomment to see the minimum of the list with the number of words per song
print(max(love_ratio_list_)) # uncomment to see the maximum of the list with the number of words per song

# Creating the regularization step for the length attribute

for artist in set(artists):   

    num_neutral = 0
    num_negative = 0
    num_positive = 0

    num_words = 0
    all_words = ''
    raw_text = "" 

    counter_for_love_words = 0
    
    for song in artist_s_cleaned_songs_list(artist):
        cleaned_song_name = song.split("cleaned_")[1].split("~")[1]
        id_ = song.split("cleaned_")[1].split("~")[0]
        f = open('Lyrics/Cleaned_Songs/' + song , 'rb')
        for sentence in f.readlines():
            this_sentence = sentence.decode('utf-8')            
            raw_text += this_sentence
        words = raw_text.split(" ")
        for item in words:
            if item in love_words_list_:
                counter_for_love_words += 1
                len_counter_for_love_words = len(love_words_list_)
                love_ratio = counter_for_love_words/float(len_counter_for_love_words)
                love_ratio = round(love_ratio,4)
                love_ratio_list_.append(love_ratio)
            
        regularization_step = (love_ratio - min_love_ratio_list_)/(max_love_ratio_list_ - min_love_ratio_list_) 
        love_ratio_score = 1*regularization_step + 0*(1-regularization_step)
        love_ratio_score = round(love_ratio_score,2)
        print(love_ratio_score) # uncomment to see the complexity of each song

[0.8095, 0.1905, 0.0, 0.2857, 0.0, 0.0, 0.1429, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2857, 0.6667, 1.0476, 1.619, 2.2381, 2.8095, 3.381, 4.0, 4.619, 5.2857, 6.0952, 7.619, 9.1905, 10.7619, 12.3333, 13.9524, 15.619, 17.3333, 19.2857, 0.0952, 1.0, 0.0, 0.0476, 0.0, 0.0476, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0952, 0.0, 0.0, 0.1905, 0.5238, 1.0952, 0.0, 0.2857, 0.0952, 0.2381, 0.619, 1.0476, 0.1429, 0.0, 0.1429, 0.1429, 0.0, 0.2857, 0.0476, 0.0, 0.1429, 0.4286, 0.0, 0.2381, 0.3333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2857, 0.0, 0.0, 0.0, 0.0, 0.0476, 0.0476, 0.0476, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1429, 0.2857, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3333, 0.0, 0.0, 0.0, 0.0476, 0.0952, 0.0, 0.1429, 0.5238, 1.0952, 0.0476, 0.1905, 0.381, 0.0, 0.0, 0.0, 0.0476, 0.0, 0.0, 0.0, 0.2381, 0.0476, 1.4286, 0.0, 0.0, 0.1429, 0.0, 0.0, 0.0, 0.1429, 0.4286, 0.0, 0.0, 0.0476, 0.0952, 0.5714, 0.0, 0.0952, 0.0, 0.0, 0.1905, 0.0476, 0.0, 0.0952, 0.0476, 0.0952, 0.2857, 0.0, 0.0, 0.0, 0.0952, 0.

In [10]:
df = pd.DataFrame(columns=('artist', 'pos', 'neu', 'neg'))
sid = SentimentIntensityAnalyzer()
i=0

os.chdir('/Users/alexpapaioannou/Dropbox/Personal Things/Studies Related/NY/COLUMBIA UNIVERSITY/IEOR/MSMSE/Courses/IEOR 4501 E 001 TOOLS FOR ANALYTICS/TOOLS_2019/Project_2019/')
        
num_neutral = 0
num_negative = 0
num_positive = 0
unique_words_list_ = []
love_words_list_ = [
                   'adore', 'adores', 'adorable', 'affection', 'amour', 'angel', 'bliss', 
#                    'care', 'caring', 'chocolate', 'companion', 'compassion', 'concern', 
#                    'darling', 'dear', 'desire', 'devotion', 'endearment', 'family', 
#                    'fondness', 'forever', 'friendship', 'fun', 'God', 'happiness', 'happy', 
                   'happily', 'heart', 'hugs', 'husband', 'infatuation', 'inspiration', 
                   'intimacy', 'joy', 'kiss', 'kisses', 
                   'love', 'loves', 'loved', 'loving', 
#                    'loyalty', 'marriage', 'passion', 'relationship', 'romance', 'sex', 
#                    'sweet', 'sweetheart', 'tenderness', 'trust', 'warmth', 'wife'
                    ]

love_ratio_list_ = []
dict_for_json = {}

with open('./data_output.txt', 'w') as f:
    print('{\n      "characterizations": [')
    print('\n             {')
    for artist in artists:   

        num_neutral = 0
        num_negative = 0
        num_positive = 0
        
        num_words = 0
        all_words = ''
        raw_text = "" 
        
        counter_for_love_words = 0


#         if len(artist_s_cleaned_songs_list(artist)) == 1:
        
        for song in artist_s_cleaned_songs_list(artist):
            cleaned_song_name = song.split("cleaned_")[1].split("~")[1]
            id_ = song.split("cleaned_")[1].split("~")[0]
            print(f'\n                     "id" = {id_},\n                     "artist" = {artist},') 

            f = open('Lyrics/Cleaned_Songs/' + song , 'rb')
            for sentence in f.readlines():
                this_sentence = sentence.decode('utf-8')
                
                raw_text += this_sentence
                
                num_words_this = len(this_sentence.split(" "))
                num_words += num_words_this
                
                comp = sid.polarity_scores(this_sentence)
                comp = comp['compound']
                if comp >= 0.5:
                    num_positive += 1
                elif comp > 0 and comp < 0.5:
                    num_neutral += 1
                else:
                    num_negative += 1
                
            sentence_for_profanity_check_ = [this_sentence]
            profanity_check = predict_prob(sentence_for_profanity_check_)
            profanity_score = float(' '.join(map(str, profanity_check)))
                
            regularization_step = (num_words - min_num_words_list_)/(max_num_words_list_ - min_num_words_list_) 
            num_words_score = 1*regularization_step + 0*(1-regularization_step)
            num_words_score = round(num_words_score,2)

            num_total = num_negative + num_neutral + num_positive
            if num_total != 0:
                percent_negative = round((num_negative/float(num_total))*100,2)
                percent_neutral = round((num_neutral/float(num_total))*100,2)
                percent_positive = round((num_positive/float(num_total))*100,2)
                df.loc[i] = (artist, percent_positive, percent_neutral, percent_negative)
                i+=1
            else:
                percent_negative = 0
                percent_neutral = 0
                percent_positive = 0
            
            words = raw_text.split(" ")
            filtered_words = [word for word in words if word not in stopwords.words('english') and len(word) > 1 and word not in ['na','la']] # remove the stopwords
            
            a = len(set(filtered_words))
            b = len(words)
            unique_words_ratio = a/float(b)
            unique_words_ratio = round(unique_words_ratio,2)
            unique_words_list_.append(unique_words_ratio)
            
            
            for item in words:
                if item in love_words_list_:
                    counter_for_love_words += 1
            len_counter_for_love_words = len(love_words_list_)
            love_ratio = counter_for_love_words/float(len_counter_for_love_words)
            love_ratio = round(love_ratio,4)
            love_ratio_list_.append(love_ratio) 
            
            regularization_step = (love_ratio - min_love_ratio_list_)/(max_love_ratio_list_ - min_love_ratio_list_) 
            love_ratio_score = 1*regularization_step + 0*(1-regularization_step)
            love_ratio_score = round(love_ratio_score,2)
            
            df.loc[i] = (artist, percent_positive, percent_neutral, percent_negative)
            i+=1
            print(f'                     "title" = {cleaned_song_name},\n                     "kid_safe" = {{:.1f}}'.format(profanity_score))
            print(f'                     "love" = {{:.1f}}'.format(love_ratio_score))
            print(f'                     "mood" = {{:.1f}}'.format(comp))
            print(f'                     "length" = {{:.1f}}'.format(num_words_score))
            print(f'                     "complexity" = {{:.1f}}'.format(unique_words_ratio))
            print('             },\n')
            print('      ]\n')
            print('}')

#         else:        
#             for song in artist_s_cleaned_songs_list(artist):
#                 cleaned_song_name = song.split("cleaned_")[1].split("~")[1]
#                 id_ = song.split("cleaned_")[1].split("~")[0]
#                 print(f' "id" = {id_},\n') 

#             f = open('Lyrics/Cleaned_Songs/' + song , 'rb')
#             for sentence in f.readlines():
#                 this_sentence = sentence.decode('utf-8')
#                 comp = sid.polarity_scores(this_sentence)
#                 comp = comp['compound']
#                 if comp >= 0.5:
#                     num_positive += 1
#                 elif comp > -0.5 and comp < 0.5:
#                     num_neutral += 1
#                 else:
#                     num_negative += 1
#                 sentence_for_profanity_check_ = [this_sentence]
#                 profanity_check = predict_prob(sentence_for_profanity_check_)
#                 profanity_score = float(' '.join(map(str, profanity_check)))

#             num_total = num_negative + num_neutral + num_positive
#             if num_total != 0:
#                 percent_negative = round((num_negative/float(num_total))*100,2)
#                 percent_neutral = round((num_neutral/float(num_total))*100,2)
#                 percent_positive = round((num_positive/float(num_total))*100,2)
#                 df.loc[i] = (artist, percent_positive, percent_neutral, percent_negative)
#                 i+=1
#             else:
#                 percent_negative = 0
#                 percent_neutral = 0
#                 percent_positive = 0
#                 df.loc[i] = (artist, percent_positive, percent_neutral, percent_negative)
#                 i+=1

#             # find the max between percent_negative, percent_neutral, percent_positive
#             if max(percent_negative, percent_neutral, percent_positive) == percent_negative:
#                 mood = 'negative'
#             elif max(percent_negative, percent_neutral, percent_positive) == percent_neutral:
#                 mood = 'neutral'
#             elif percent_negative == 0 and percent_neutral == 0 and percent_positive == 0:
#                 mood = 'indifferent'
#             else:
#                 mood = 'positive'

#             print(f' "title" = {cleaned_song_name},\n "kid_safe" = {{:.1f}}'.format(profanity_score))
#             print(f' "love" = ,\n "mood" = {mood},\n')

{
      "characterizations": [

             {

                     "id" = 424,
                     "artist" = Reel Big Fish,
                     "title" = We Care,
                     "kid_safe" = 0.1
                     "love" = 0.0
                     "mood" = 0.6
                     "length" = 0.1
                     "complexity" = 0.3
             },

      ]

}

                     "id" = 122,
                     "artist" = Richard "Cookie" Thomas,
                     "title" = Hallelujah I Just Love Her So,
                     "kid_safe" = 0.1
                     "love" = 0.0
                     "mood" = 0.8
                     "length" = 0.0
                     "complexity" = 0.4
             },

      ]

}

                     "id" = 117,
                     "artist" = Spike Jones & His City Slickers,
                     "title" = Clink, Clink, Another Drink,
                     "kid_safe" = 0.1
                     "love" = 0.0
                     "mood" 

In [18]:
# def love_ratio_score(song)->float:
    
#     words = []
#     raw_text = "" 
#     counter_for_love_words = 0
    
#     love_words_list_ = [
#                    'adore', 'adores', 'adorable', 'affection', 'amour', 'angel', 'bliss', 
# #                    'care', 'caring', 'chocolate', 'companion', 'compassion', 'concern', 
# #                    'darling', 'dear', 'desire', 'devotion', 'endearment', 'family', 
# #                    'fondness', 'forever', 'friendship', 'fun', 'God', 'happiness', 'happy', 
#                    'happily', 'heart', 'hugs', 'husband', 'infatuation', 'inspiration', 
#                    'intimacy', 'joy', 'kiss', 'kisses', 
#                    'love', 'loves', 'loved', 'loving', 
# #                    'loyalty', 'marriage', 'passion', 'relationship', 'romance', 'sex', 
# #                    'sweet', 'sweetheart', 'tenderness', 'trust', 'warmth', 'wife'
#                     ]
    
#     love_ratio_list_ = []

#     f = open('Lyrics/Cleaned_Songs/' + song , 'rb')
#     for sentence in f.readlines():
#         this_sentence = sentence.decode('utf-8')
#         raw_text += this_sentence

#     words = raw_text.split(" ")
#     filtered_words = [word for word in words if word not in stopwords.words('english') and len(word) > 1 and word not in ['na','la']] # remove the stopwords

#     for item in words:
#         if item in love_words_list_:
#             counter_for_love_words += 1
#     len_counter_for_love_words = len(love_words_list_)
#     love_ratio = counter_for_love_words/float(len_counter_for_love_words)
#     love_ratio = round(love_ratio,4)
#     love_ratio_list_.append(love_ratio)
    
#     for artist in set(artists):   
        
#         for song in artist_s_cleaned_songs_list(artist):
#             cleaned_song_name = song.split("cleaned_")[1].split("~")[1]
#             id_ = song.split("cleaned_")[1].split("~")[0]
#             f = open('Lyrics/Cleaned_Songs/' + song , 'rb')
#             for sentence in f.readlines():
#                 this_sentence = sentence.decode('utf-8')
                
#                 raw_text += this_sentence
            
#             words = raw_text.split(" ")
#             filtered_words = [word for word in words if word not in stopwords.words('english') and len(word) > 1 and word not in ['na','la']] # remove the stopwords
            
#             for item in words:
#                 if item in love_words_list_:
#                     counter_for_love_words += 1
#             len_counter_for_love_words = len(love_words_list_)
#             love_ratio = counter_for_love_words/float(len_counter_for_love_words)
#             love_ratio = round(love_ratio,4)
#             love_ratio_list_.append(love_ratio) 
            
#     min_love_ratio_list_ = min(love_ratio_list_)
#     max_love_ratio_list_ = max(love_ratio_list_)

#     regularization_step = (love_ratio - min_love_ratio_list_)/(max_love_ratio_list_ - min_love_ratio_list_) 
#     love_ratio_score = 1*regularization_step + 0*(1-regularization_step)
#     love_ratio_score = round(love_ratio_score,1)
#     return love_ratio_score

In [ ]:
# a = love_ratio_score('cleaned_688~I Wanna Be Loved~Buy This Song.txt')
# print(a)